In [ ]:
from pyiron import Project

In [ ]:
pr = Project("submit")

In [ ]:
active_cores = 0
number_of_cores = 2

In [ ]:
job_dict = {}
job_next_lst = []
while True:
    continue_check = True
    if len(job_next_lst) == 0:
        df = pr.job_table()
        job_next_lst = df[
            (df["status"] == "submitted") & 
            (df["hamilton"] == "Lammps")
        ].id.values
    for i, job_id in enumerate(job_next_lst):
        if job_id not in job_dict:
            job = pr.load(job_id)
            job.status.created = True 
            job.server.run_mode.thread = True
            job_cores = job.server.cores
            active_cores += job_cores
            job.run()
            job_dict[job_id] = [job_cores, job._process]
        if active_cores == number_of_cores:
            job_next_lst = job_next_lst[i+1:]
            break

    while continue_check:
        finhshed_jobs = []
        for k, [c, p] in job_dict.items():
            if not p.is_alive(): 
                finhshed_jobs.append(k)
                active_cores -= c
                continue_check = False
        for k in finhshed_jobs:
            del job_dict[k]